In [1]:

import findspark
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')

In [2]:
import pandas as pd
import pyspark
from pyspark.sql import *

In [3]:
from pyspark.ml.feature  import Tokenizer,RegexTokenizer,StopWordsRemover,CountVectorizer,OneHotEncoderEstimator,StringIndexer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import *
from pyspark.ml.classification import LogisticRegression

In [4]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors
from pyspark.ml.feature import Word2Vec
import pyspark.sql.functions as f
from pyspark.ml.feature import Word2VecModel

In [5]:
sc=pyspark.SparkContext()

In [6]:
spark= SparkSession \
       .builder \
       .appName("DICPA3") \
       .config("spark.some.config.option","some-value") \
       .getOrCreate() 

In [7]:
train_pd = pd.read_csv('train.csv')

In [8]:
mapping_pd = pd.read_csv('mapping.csv')


In [9]:
mapping=spark.createDataFrame(mapping_pd)


In [10]:
train=spark.createDataFrame(train_pd)

In [11]:
train.show()

+--------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|
+--------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov, a hard-w...|['World cinema', ...|
|31186339|    The Hunger Games|The nation of Pan...|['Action/Adventur...|
|20663735|          Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|          ['Comedy']|
|  595909|   A Cry in the Dark|Seventh-day Adven...|['Crime Fiction',...|
| 5272176|            End Game|The president is ...|['Action/Adventur...|
| 1952976|          Dark Water|{{plot}} The film...|['Thriller', 'Dra...|
|24225279|                Sing|The story begins ...|           ['Drama']|
| 2462689|       Meet John Doe|Infuriated at bei...|['Black-and-white...|
|20532852|Destination Meatball|A line of people ...|['Animation', 'Sh...|
|15401493|    Husband for Hire|Lola  a

In [12]:
regexTokenizer = RegexTokenizer(inputCol="plot", outputCol="words", pattern="\\W")


train=regexTokenizer.transform(train)

stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered")

train=stopwordsRemover.transform(train)
####################################################################################################
# word2Vectrain = Word2Vec(vectorSize=1500, inputCol="filtered", outputCol="features" ,maxIter=1)

# modeltrain = word2Vectrain.fit(train)
# modeltrain.save('./part4trainword2vec1500')
####################################################################################################
# word2Vectrain = Word2Vec(vectorSize=500, inputCol="filtered", outputCol="features" ,maxIter=1)

# modeltrain = word2Vectrain.fit(train)
# modeltrain.save('./part4trainword2vec500')
####################################################################################################

In [13]:

#trainmodel = Word2VecModel.load('./part4trainword2vec1500')
trainmodel = Word2VecModel.load('./part4trainword2vec500')
train = trainmodel.transform(train)

In [14]:
mapping=spark.createDataFrame(mapping_pd,['id',"genre"])
mapgenre=mapping.select("genre","id").rdd.collectAsMap()

In [15]:
mapping.show()
count=mapping.count()

+---+----------------+
| id|           genre|
+---+----------------+
|  0|           Drama|
|  1|          Comedy|
|  2|    Romance Film|
|  3|        Thriller|
|  4|          Action|
|  5|    World cinema|
|  6|   Crime Fiction|
|  7|          Horror|
|  8| Black-and-white|
|  9|           Indie|
| 10|Action/Adventure|
| 11|       Adventure|
| 12|     Family Film|
| 13|      Short Film|
| 14|  Romantic drama|
| 15|       Animation|
| 16|         Musical|
| 17| Science Fiction|
| 18|         Mystery|
| 19| Romantic comedy|
+---+----------------+



In [16]:

train.show()


+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|               words|            filtered|            features|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov, a hard-w...|['World cinema', ...|[shlykov, a, hard...|[shlykov, hard, w...|[-0.0258779611767...|
|31186339|    The Hunger Games|The nation of Pan...|['Action/Adventur...|[the, nation, of,...|[nation, panem, c...|[0.01190575840073...|
|20663735|          Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|[poovalli, induch...|[poovalli, induch...|[-0.0049900839706...|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|          ['Comedy']|[the, lemon, drop...|[lemon, drop, kid...|[-0.0023341972278...|
|  595909|   A Cry in the Dark|Seventh-da

In [17]:

def function1(g):
    x=[]
    genre=g[1:-1].split(",")  
    for i in genre:
        value=i.strip()[1:-1]
        x.append(mapgenre.get(value))
    x.sort()
    return x
myfunction1=f.udf(function1,ArrayType(IntegerType()))
train=train.withColumn("index",myfunction1("genre"))

train.select('index').show()

+----------------+
|           index|
+----------------+
|          [0, 5]|
|  [0, 4, 10, 17]|
|      [0, 4, 16]|
|             [1]|
|       [0, 5, 6]|
|   [0, 3, 4, 10]|
|       [0, 3, 7]|
|             [0]|
|[0, 1, 2, 8, 19]|
|    [12, 13, 15]|
|             [1]|
|    [0, 1, 5, 6]|
|             [1]|
|             [1]|
|             [7]|
|   [3, 6, 7, 18]|
|             [0]|
| [2, 3, 4, 6, 9]|
|          [0, 9]|
|             [0]|
+----------------+
only showing top 20 rows



In [18]:

def function2(g):
    y=1
    x=[]
    for i in range(0,count):
        x.append(0)
    
    
    for i in g:
        x[i]=1
        
    
    return x
myfunction2=f.udf(function2,ArrayType(IntegerType()))
traindata=train.withColumn("label",myfunction2("index"))
traindata.select('features','index','label').show()

+--------------------+----------------+--------------------+
|            features|           index|               label|
+--------------------+----------------+--------------------+
|[-0.0258779611767...|          [0, 5]|[1, 0, 0, 0, 0, 1...|
|[0.01190575840073...|  [0, 4, 10, 17]|[1, 0, 0, 0, 1, 0...|
|[-0.0049900839706...|      [0, 4, 16]|[1, 0, 0, 0, 1, 0...|
|[-0.0023341972278...|             [1]|[0, 1, 0, 0, 0, 0...|
|[-0.0144077781772...|       [0, 5, 6]|[1, 0, 0, 0, 0, 1...|
|[-0.0054802323886...|   [0, 3, 4, 10]|[1, 0, 0, 1, 1, 0...|
|[-0.0233243806883...|       [0, 3, 7]|[1, 0, 0, 1, 0, 0...|
|[3.82496744674270...|             [0]|[1, 0, 0, 0, 0, 0...|
|[-0.0091316081517...|[0, 1, 2, 8, 19]|[1, 1, 1, 0, 0, 0...|
|[0.01386682893114...|    [12, 13, 15]|[0, 0, 0, 0, 0, 0...|
|[-0.0061433933152...|             [1]|[0, 1, 0, 0, 0, 0...|
|[0.01378680495968...|    [0, 1, 5, 6]|[1, 1, 0, 0, 0, 1...|
|[0.00439556976647...|             [1]|[0, 1, 0, 0, 0, 0...|
|[0.00599432231136...|  

In [19]:
test_pd = pd.read_csv('test.csv')
test=spark.createDataFrame(test_pd)
test.show()

+--------+--------------------+--------------------+
|movie_id|          movie_name|                plot|
+--------+--------------------+--------------------+
| 1335380|              Exodus|The film is based...|
|29062594|A la salida nos v...|A group of teenag...|
| 9252321|   Come Back, Africa|This story of a Z...|
|13455076|       A Merry Mixup|The Stooges play ...|
|24165951|        Getting Even|A soldier-of-fort...|
| 1925869|  River of No Return|Set in the Northw...|
|10799612|          Amici miei|Like in many othe...|
|28238240|Mickey's Big Game...|Mickey and the Sc...|
|17124781|The Good, the Bad...|In the desert wil...|
|28207941|    The Dancing Fool|Bimbo and Koko ar...|
|19174305|              Tahaan|Tahaan  lives wit...|
|18392317|     Mysterious Mose|Betty is startled...|
|34420857|Kelviyum Naane Pa...|Nirmal ([[Karthik...|
| 4039635|   First on the Moon|A group of journa...|
| 8034072|  Journey of a Woman|Vaibhavari Sahay,...|
| 4016437|     Sophie's Choice|In 1947, the mo

In [20]:



test=regexTokenizer.transform(test)



test=stopwordsRemover.transform(test)





In [21]:
####################################################################################################
# word2Vectest = Word2Vec( vectorSize=1500,inputCol="filtered", outputCol="features" ,maxIter=1)

# modeltest= word2Vectest.fit(test)
# modeltest.save('./part4testword2vec1500')
####################################################################################################
# word2Vectest = Word2Vec( vectorSize=500,inputCol="filtered", outputCol="features" ,maxIter=1)

# modeltest= word2Vectest.fit(test)
# modeltest.save('./part4testword2vec500')
####################################################################################################

In [22]:
#testmodel = Word2VecModel.load('./part4testword2vec1500')
testmodel = Word2VecModel.load('./part4testword2vec500')
testdata = testmodel.transform(test)
testdata.show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               words|            filtered|            features|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 1335380|              Exodus|The film is based...|[the, film, is, b...|[film, based, eve...|[0.00713589835897...|
|29062594|A la salida nos v...|A group of teenag...|[a, group, of, te...|[group, teenagers...|[0.00307899780427...|
| 9252321|   Come Back, Africa|This story of a Z...|[this, story, of,...|[story, zulu, fam...|[0.00808871257595...|
|13455076|       A Merry Mixup|The Stooges play ...|[the, stooges, pl...|[stooges, play, t...|[-0.0012985587395...|
|24165951|        Getting Even|A soldier-of-fort...|[a, soldier, of, ...|[soldier, fortune...|[0.00747495703035...|
| 1925869|  River of No Return|Set in the Northw...|[set, in, the, no...

In [ ]:
x=[]

for i in range(0,count):
    
    def parsePoint(line):
    
        return LabeledPoint(line.label[i], MLLibVectors.fromML(line.features))
    parsedData = traindata.rdd.map(parsePoint)
    part4model=LogisticRegressionWithLBFGS.train(parsedData)
    filename='./part4test'+str(i)
    part4model.save(spark,filename)
    newmodel = LogisticRegressionModel.load(spark,filename)
    labelsAndPreds = testdata.rdd.map(lambda p: (p.movie_id, newmodel.predict(MLLibVectors.fromML(p.features))))
    x.append(labelsAndPreds.collect())

In [ ]:
y={}
for i in x:
    for j in i:
        if j[0] in y:
            y[j[0]].append(j[1])
        else:
            y[j[0]]=[]
            y[j[0]].append(j[1])
for i in y:
    print(i,y[i])

In [ ]:
keypred={}

for i in y:
    x=""
    for j in y[i]:
        x=x+" "+str(j)
    x=x.strip()
    keypred[i]=x
for i in keypred:
    print(i,str(keypred[i]))

In [ ]:
import csv

with open ('output4.csv','w',newline="") as mfile:
    writer=csv.writer(mfile)
    writer.writerow(["movie_id","predictions"])
    for key , value in keypred.items():
        writer.writerow([key,value])